In [80]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [81]:
book_df = pd.read_csv('./club.csv', encoding = "Latin1", sep = ",")
user_df = pd.read_csv('./user.csv', encoding = "Latin1", sep = ",")
club_df = pd.read_csv('./club.csv', encoding = "Latin1", sep = ",")
club_book_df = pd.read_csv('./club_user.csv', encoding = "Latin1", sep = ",")
club_user_df = pd.read_csv('./club_user.csv', encoding = "Latin1", sep = ",")

In [82]:
club_user_age = club_user_df.merge(user_df, left_on = 'user_id', right_on = 'id')
club_user_age = club_user_age[['id_x', 'club_id', 'user_id', 'age']]

In [83]:
average_club_age_df = club_user_age.groupby(['club_id'])['age'].mean().reset_index(name='age')
average_club_age_df.describe()

,club_id,age
count,10.00000,10.000000
mean,16.50000,78.158709
std,3.02765,8.268389
min,12.00000,68.361702
25%,14.25000,70.698810
50%,16.50000,77.315000
75%,18.75000,84.786782
max,21.00000,89.574074


In [84]:
# Perform a many-to-many merge so that we can later check what books a user and club have in common

#club_book_user = club_book_df.merge(book_df, left_on = 'user_id', right_on = 'id')

club_book_user = \
club_df.merge(club_book_df) \
  .merge(club_df.merge(club_book_df), on='id') \
  .groupby(['name_x', 'name_y'])['id'].agg(list) \
  .reset_index()

club_book_user
#club_book_user = club_book_user[['id_x', 'club_id', 'user_id', 'fa']]


,name_x,name_y,id


In [85]:
# Return club_id of the club with the closest average age
my_age = 30
age_list = list(average_club_age_df['age'])

new_age_list = []
for i in age_list:
    new_age_list.append(i - my_age)
lstt = [abs(ele) for ele in new_age_list]
closest_age = age_list[lstt.index(min(lstt))]
club_with_closest_age = average_club_age_df.loc[average_club_age_df['age'] == closest_age, 'club_id'].reset_index(name = 'club_id')
club_with_closest_age

,index,club_id
0,9,21


In [86]:
# Return clubs in the same country
my_country = 'USA'
clubs_in_my_country = []

for index, row in club_df.iterrows():
    if my_country.lower() == row[2].lower():
        clubs_in_my_country.append(row[1] + ", " + row[2])

clubs_in_my_country

["AlexGardner's Club, USA"]

In [87]:
# Return clubs in the same city
my_city = 'Manchester'
clubs_in_my_city = []

for index, row in club_df.iterrows():
    if my_city.lower() == row[2].lower():
        clubs_in_my_city.append(row[1] + ", " + row[2])

clubs_in_my_city

["SimonHardy's Club, Manchester", "FrankAdams's Club, Manchester"]

In [88]:
# Return clubs with matching favourite books
my_favourite_books = []

In [89]:
d ={}
for i,j in enumerate(club_user_book.ISBN.unique()):
    d[j] =i
club_user_book['unique_id_book'] = club_user_book['ISBN'].map(d)


NameError: name 'club_user_book' is not defined

In [ ]:
users_clubs_pivot_matrix_df = club_user_book.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-book').fillna(0)



In [ ]:
users_clubs_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,26069,26070,26071,26072,26073,26074,26075,26076,26077,26078
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
users_clubs_pivot_matrix_df = users_clubs_pivot_matrix_df.values
users_clubs_pivot_matrix_df

array([[0., 6., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_clubs_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [ ]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [ ]:
all_user_predicted_books = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_books

array([[ 0.00000000e+00,  3.01145261e-34, -1.44341941e-34, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.58780903e-38, -3.19805495e-38, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  5.38921851e-35, -1.80280872e-34, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  1.53681908e-35, -3.57548715e-36, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.91903239e-34, -5.66829937e-36, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.00020459e-35, -1.05741351e-34, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [ ]:
def top_cosine_similarity(data, club_id, top_n=10):
    index = club_id 
    club_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(club_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_clubs(club_user_book, club_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    club_user_book[club_user_book.unique_id_club == club_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(club_user_book[club_user_book.unique_id_club == id]['Book-Title'].values[0])



In [ ]:
k = 50
movie_id =25954  
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_clubs(club_user_book, 25954, top_cosine_similarity(sliced, movie_id, top_n))


Recommendations for Pulse Points: 

The Witchfinder (Amos Walker Mystery Series)
Alone in a Crowd (Harper Monogram)
Jackie Oh


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


# App link:

https://book-recommendation-system-svd.herokuapp.com/

